## Visualization of temperature in Berlin-Mitte

In [ ]:
#!pip install geopandas==0.8.0
#!pip install folium

In [1]:
import pandas as pd 
import folium
import re
import geopandas as gpd
import random
import plotly.express as px

In [24]:
df = pd.read_csv('/home/lorena/Documents/bootcamp/W05/data/ECA_blended_custom/TG_STAID004563.txt', skiprows=18, skipinitialspace=True)
df['TG'] = df['TG'] * 0.1
df['DATE'] = pd.to_datetime(df['DATE'].astype(str), format="%Y-%m-%d")
df.set_index(df['DATE'], inplace=True)
df = df[['DATE','TG']]

In [25]:
df_berlin_yearly = df.groupby(df.index.year)['TG'].mean()
df_berlin_yearly = pd.DataFrame(df_berlin_yearly)
df_berlin_yearly['city'] = 'Berlin'
df_berlin_yearly['country'] = 'Germany'
df_berlin_yearly['iso'] = 'DEU'
df_berlin_yearly['district'] = 'Mitte'
df_berlin_yearly['year'] = df_berlin_yearly.index

In [13]:
#Source: https://github.com/m-hoerz/berlin-shapes

berlin_bezirke = '/home/lorena/Documents/bootcamp/W05/data/berliner-bezirke.geojson'
geojson = open(berlin_bezirke).read()
districts = re.findall(r'spatial_alias": "([^,]+)",', geojson)

berlin_mitte = folium.Map(location=[52.54, 13.36],zoom_start=10,tiles='CartoDB positron')

berlin_mitte.choropleth(
    geo_data=berlin_bezirke,
    name='chloropleth',
    data=df_berlin_yearly,
    columns=['district', 'TG'],
    key_on='properties.spatial_alias',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Temperature in degrees Celsius'
)

folium.LayerControl().add_to(berlin_mitte)
berlin_mitte